# Import Library


In [1]:
import pandas as pd
import numpy as np

# Load Data

In [2]:
df_users = pd.read_csv("clean_users.csv")
df_tracking = pd.read_csv("clean_tracking.csv")
df_submission = pd.read_csv("clean_submissions.csv")
df_complete = pd.read_csv("clean_completions.csv")
df_regist = pd.read_csv("clean_exam_registrations.csv")
df_exam = pd.read_csv("clean_exam_results.csv")

In [3]:
for name, df in {
    "Users": df_users,
    "Tracking": df_tracking,
    "Submission": df_submission,
    "Completion": df_complete,
    "Registration": df_regist,
    "Exam Results": df_exam
}.items():

    print(f"\n===== {name} =====")
    print(df.info())



===== Users =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            31 non-null     int64  
 1   display_name  31 non-null     object 
 2   name          31 non-null     object 
 3   email         31 non-null     object 
 4   phone         30 non-null     float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.3+ KB
None

===== Tracking =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101736 entries, 0 to 101735
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               101736 non-null  int64 
 1   journey_id       101736 non-null  int64 
 2   tutorial_id      101736 non-null  int64 
 3   developer_id     101736 non-null  int64 
 4   status           101736 non-null  int64 
 5   last_viewed      11885 non-null   object
 6   first_

# Normalisasi Primary Key untuk Tahap Merging

In [4]:
df_users.rename(columns={"id": "user_id"}, inplace=True)
df_tracking.rename(columns={"developer_id": "user_id"}, inplace=True)
df_submission.rename(columns={"submitter_id": "user_id"}, inplace=True)
df_complete.rename(columns={"user_id": "user_id"}, inplace=True)
df_regist.rename(columns={"examinees_id": "user_id"}, inplace=True)
df_exam.rename(columns={"exam_registration_id": "regist_id"}, inplace=True)

# SUmmary agg setiap user

## Tracking Summary

In [5]:
tracking_summary = df_tracking.groupby("user_id").agg({
    "tutorial_id": "count",
    "completed_at": lambda x: x.notna().sum()
}).rename(columns={
    "tutorial_id": "total_tracking_events",
    "completed_at": "total_completed_modules"
})


## Submission SUmmary

In [6]:
# Convert rating column to numeric
df_submission["rating"] = pd.to_numeric(df_submission["rating"], errors="coerce")

# Create aggregated summary
submission_summary = df_submission.groupby("user_id").agg({
    "id": "count",
    "rating": "mean"
}).rename(columns={
    "id": "total_submissions",
    "rating": "avg_submission_rating"
})


## Completion SUmmary

In [7]:
df_complete["study_duration"] = pd.to_numeric(df_complete["study_duration"], errors="coerce")

df_complete["avg_submission_rating"] = pd.to_numeric(df_complete["avg_submission_rating"], errors="coerce")


In [8]:
completion_summary = df_complete.groupby("user_id").agg({
    "study_duration": "mean",
    "avg_submission_rating": "mean"
}).rename(columns={
    "study_duration": "avg_study_duration",
    "avg_submission_rating": "avg_completion_rating"
})


In [9]:
df_complete["study_duration"].describe()


,study_duration
count,1027.000000
mean,65.194742
std,145.054789
min,0.000000
25%,2.500000
50%,15.000000
75%,56.000000
max,993.000000


## Exam Score Summary

In [10]:
# Ensure matching dtype
df_exam["regist_id"] = df_exam["regist_id"].astype(str)
df_regist["id"] = df_regist["id"].astype(str)

# Convert numeric columns
df_exam["score"] = pd.to_numeric(df_exam["score"], errors="coerce")
df_exam["is_passed"] = pd.to_numeric(df_exam["is_passed"], errors="coerce")

In [11]:
df_exam_matched = df_exam.merge(
    df_regist,
    left_on="created_at",
    right_on="exam_finished_at",
    how="left"
)


In [12]:
# Convert numeric
df_exam_matched["score"] = pd.to_numeric(df_exam_matched["score"], errors="coerce")
df_exam_matched["is_passed"] = pd.to_numeric(df_exam_matched["is_passed"], errors="coerce")

# Aggregate per user
exam_summary = df_exam_matched.groupby("user_id").agg({
    "score": "mean",
    "is_passed": "mean"
}).rename(columns={
    "score": "avg_exam_score",
    "is_passed": "exam_pass_rate"
}).reset_index()

exam_summary.head()


,user_id,avg_exam_score,exam_pass_rate
0,3390.0,79.804382,0.721440
1,5774.0,67.020000,0.518000
2,11836.0,78.500000,0.759563
3,17833.0,85.950867,0.835260
4,32258.0,84.335052,0.807990


In [13]:
exam_summary["user_id"] = exam_summary["user_id"].astype("Int64")


## Save Summary

In [14]:
exam_summary.to_csv("summary_exam.csv", index=False)
tracking_summary.to_csv("summary_tracking.csv", index=False)
submission_summary.to_csv("summary_submission.csv", index=False)
completion_summary.to_csv("summary_completion.csv", index=False)

print("Saved all summary tables.")


Saved all summary tables.


# Menggabunbgkan Data

In [15]:
import pandas as pd

df_users = pd.read_csv("clean_users.csv")
tracking_summary = pd.read_csv("summary_tracking.csv")
submission_summary = pd.read_csv("summary_submission.csv")
completion_summary = pd.read_csv("summary_completion.csv")
exam_summary = pd.read_csv("summary_exam.csv")


In [16]:
for name, df in {
    "Users": df_users,
    "Tracking": tracking_summary,
    "Submission": submission_summary,
    "Completion": completion_summary,
    "Exam": exam_summary,
}.items():
    print(name, " → Has user_id:", "user_id" in df.columns)


Users  → Has user_id: False
Tracking  → Has user_id: False
Submission  → Has user_id: False
Completion  → Has user_id: False
Exam  → Has user_id: True


In [17]:
for name, df in {
    "Users": pd.read_csv("clean_users.csv"),
    "Tracking": pd.read_csv("clean_tracking.csv"),
    "Submission": pd.read_csv("summary_submission.csv"),
    "Completion": pd.read_csv("summary_completion.csv"),
    "Exam": pd.read_csv("summary_exam.csv"),
}.items():
    print(f"\n==== {name} Columns ====")
    print(df.columns.tolist())



==== Users Columns ====
['id', 'display_name', 'name', 'email', 'phone']

==== Tracking Columns ====
['id', 'journey_id', 'tutorial_id', 'developer_id', 'status', 'last_viewed', 'first_opened_at', 'completed_at']

==== Submission Columns ====
['total_submissions', 'avg_submission_rating']

==== Completion Columns ====
['avg_study_duration', 'avg_completion_rating']

==== Exam Columns ====
['user_id', 'avg_exam_score', 'exam_pass_rate']


In [18]:
df_users.rename(columns={"id": "user_id"}, inplace=True)


In [19]:
df_tracking.rename(columns={"developer_id": "user_id"}, inplace=True)


In [20]:
# Ambil daftar user real
user_list = df_users["user_id"]

# Create indexed copies
submission_summary["user_id"] = user_list
completion_summary["user_id"] = user_list
tracking_summary["user_id"] = user_list


In [21]:
for df in [df_users, tracking_summary, submission_summary, completion_summary, exam_summary]:
    df["user_id"] = df["user_id"].astype(int)


In [22]:
df_users.to_csv("clean_users_fixed.csv", index=False)
tracking_summary.to_csv("summary_tracking_fixed.csv", index=False)
submission_summary.to_csv("summary_submission_fixed.csv", index=False)
completion_summary.to_csv("summary_completion_fixed.csv", index=False)
exam_summary.to_csv("summary_exam_fixed.csv", index=False)


In [23]:
df_final = (
    df_users
    .merge(tracking_summary, on="user_id", how="left")
    .merge(submission_summary, on="user_id", how="left")
    .merge(completion_summary, on="user_id", how="left")
    .merge(exam_summary, on="user_id", how="left")
)

print(df_final.shape)
df_final.head()


(31, 13)


,user_id,display_name,name,email,phone,total_tracking_events,total_completed_modules,total_submissions,avg_submission_rating,avg_study_duration,avg_completion_rating,avg_exam_score,exam_pass_rate
0,96989,igihcksn,Inggih Wicaksono,igihcksn@gmail.com,6.281271e+12,5526,296,133.0,1.639098,49.963636,3.967037,70.817175,0.587258
1,938276,nurrizkiadip,Nur Rizki Adi Prasetyo,nrizki@dicoding.com,6.287795e+12,2628,258,5.0,1.400000,70.666667,3.000000,84.150838,0.832402
2,5021477,rifath_2SXp,rifath,rifathali088@gmail.com,6.289662e+12,3272,477,32.0,2.250000,157.600000,4.100000,83.086957,0.826087
3,5044844,ledis_idola_h8Ge,LEDIS IDOLA,221113142@students.mikroskil.ac.id,6.282312e+12,3813,244,77.0,3.077922,45.536585,4.423750,91.056075,0.962617
4,5051374,fkaslana,Fircan Ferdinand,kaslanafircan@gmail.com,6.288222e+12,5943,321,146.0,2.383562,70.382353,3.743947,69.371429,0.657143


In [24]:
print("Unique users in final:", df_final["user_id"].nunique())
print("Rows:", len(df_final))


Unique users in final: 31
Rows: 31


In [25]:
completion_summary.head()


,avg_study_duration,avg_completion_rating,user_id
0,49.963636,3.967037,96989
1,70.666667,3.000000,938276
2,157.600000,4.100000,5021477
3,45.536585,4.423750,5044844
4,70.382353,3.743947,5051374


In [26]:
df_final[["user_id","avg_study_duration"]].head()


,user_id,avg_study_duration
0,96989,49.963636
1,938276,70.666667
2,5021477,157.600000
3,5044844,45.536585
4,5051374,70.382353


In [27]:
cols_to_round = [
    "avg_submission_rating",
    "avg_study_duration",
    "avg_completion_rating",
    "avg_exam_score",
    "exam_pass_rate"
]

df_final[cols_to_round] = df_final[cols_to_round].round(2)


In [28]:
# df_final["avg_study_duration"] = df_final["avg_study_duration"].round(2)
# df_final["avg_completion_rating"] = df_final["avg_completion_rating"].round(2)
# df_final["avg_submission_rating"] = df_final["avg_submission_rating"].round(2)
# df_final["avg_exam_score"] = df_final["avg_exam_score"].round(2)
# df_final["exam_pass_rate"] = df_final["exam_pass_rate"].round(2)


In [32]:
df_final['total_submissions'] = df_final['total_submissions'].astype('Int64')

df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   user_id                  31 non-null     int64  
 1   display_name             31 non-null     object 
 2   name                     31 non-null     object 
 3   email                    31 non-null     object 
 4   phone                    30 non-null     object 
 5   total_tracking_events    31 non-null     int64  
 6   total_completed_modules  31 non-null     int64  
 7   total_submissions        26 non-null     Int64  
 8   avg_submission_rating    26 non-null     float64
 9   avg_study_duration       29 non-null     float64
 10  avg_completion_rating    26 non-null     float64
 11  avg_exam_score           29 non-null     float64
 12  exam_pass_rate           29 non-null     float64
dtypes: Int64(1), float64(5), int64(3), object(4)
memory usage: 3.3+ KB


In [34]:
import numpy as np

def format_phone_number(phone_num):
    if pd.isna(phone_num):
        return None
    # Convert to integer first to remove scientific notation, then to string
    return str(int(phone_num))

df_final['phone'] = df_final['phone'].apply(format_phone_number)

display(df_final.head())

,user_id,display_name,name,email,phone,total_tracking_events,total_completed_modules,total_submissions,avg_submission_rating,avg_study_duration,avg_completion_rating,avg_exam_score,exam_pass_rate
0,96989,igihcksn,Inggih Wicaksono,igihcksn@gmail.com,6281270768151,5526,296,133,1.64,49.96,3.97,70.82,0.59
1,938276,nurrizkiadip,Nur Rizki Adi Prasetyo,nrizki@dicoding.com,6287794555845,2628,258,5,1.40,70.67,3.00,84.15,0.83
2,5021477,rifath_2SXp,rifath,rifathali088@gmail.com,6289662167888,3272,477,32,2.25,157.60,4.10,83.09,0.83
3,5044844,ledis_idola_h8Ge,LEDIS IDOLA,221113142@students.mikroskil.ac.id,6282312102689,3813,244,77,3.08,45.54,4.42,91.06,0.96
4,5051374,fkaslana,Fircan Ferdinand,kaslanafircan@gmail.com,6288221935189,5943,321,146,2.38,70.38,3.74,69.37,0.66


In [33]:
df_final.to_csv("final_dataset.csv", index=False)
print("📁 Saved: final_dataset.csv (shape:", df_final.shape, ")")


📁 Saved: final_dataset.csv (shape: (31, 13) )
